In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dmt.a_leadtime_sto_ambient_bs_internal_rpt
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
AS
WITH sto_data AS(
  SELECT DISTINCT
    a.po_doc,
    a.po_type,
    a.po_item,
    a.store_id,
    a.product_id,
    a.dc_id,
    a.creation_date sto_creation_date,
    a1.sto_number,
    MAX(gi_to_date) gi_date,
    SUM(po_qty * (COALESCE(b.numerator, 1) / CASE WHEN COALESCE(b.denomintr, 1) = 0 THEN 1 ELSE COALESCE(b.denomintr, 1) END)) AS sto_qty_base,
    SUM(po_gi_qty * (COALESCE(b.numerator, 1) / CASE WHEN COALESCE(b.denomintr, 1) = 0 THEN 1 ELSE COALESCE(b.denomintr, 1) END)) AS gi_qty_base

  FROM
    {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_unit b ON a.product_id = b.product_id AND a.po_unit = b.unit
    LEFT JOIN (SELECT DISTINCT sto_number FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_allocation_table) a1 ON a.po_doc = a1.sto_number
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.dc_id = c.store_id
  WHERE
    calday >= (date_sub(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS, (dayofweek(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) + 6) % 7)  - INTERVAL 26 WEEK)
    AND c.business_unit = '1100'
    AND a.po_type <> 'Collective PO'
    AND (a1.sto_number IS NULL)
  GROUP BY 1,2,3,4,5,6,7,8
),
bs AS (
  SELECT
    a.po_doc sto_number,
    a.po_item sto_item,
    a.store_id,
    a.product_id,
    a.sto_creation_date start_date,
    a.gi_date end_date,
    a.dc_id,
    'BS' sto_type,
    SUM(sto_qty_base) sto_qty,
    SUM(gi_qty_base) gi_qty
  FROM sto_data a
  GROUP BY 1,2,3,4,5,6,7,8
),
temp_wk AS (
  SELECT DISTINCT
    sto_type,
    sto_number,
    sto_item,
    product_id,
    start_date,
    end_date,
    store_id,
    a.dc_id,
    sto_qty,
    gi_qty,
    COUNT(IF(f.workingday = 0,1,NULL)) subtract_wkd,
    COUNT(IF(g.dc_id IS NOT NULL and f.date = g.stock_count_date,1,NULL)) subtract_stockcnt
  FROM
    bs a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_working_day_cnt f ON a.end_date >= f.date AND a.start_date <= f.date
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dmt.d_portal_stock_count_date_bydc g ON a.end_date >= g.stock_count_date AND a.start_date <= g.stock_count_date AND a.dc_id = CAST(g.dc_id AS STRING)
  GROUP BY 1,2,3,4,5,6,7,8,9,10
),
ontime_check AS (
  SELECT
    a.*,
    DATEDIFF(end_date, start_date) - (IFNULL(subtract_wkd, 0) + IFNULL(subtract_stockcnt, 0)) leadtime_actual,
    CASE WHEN city IN ('HÀ NỘI','HỒ CHÍ MINH') AND manufacturer_id = '1000' THEN 4 ELSE 7 END leadtime_qual,
    DATEDIFF(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), start_date)
 - (COALESCE(subtract_wkd, 0) + COALESCE(subtract_stockcnt, 0)) AS time_tilltoday
  FROM
    temp_wk a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store USING (store_id)
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product USING (product_id)
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time d ON a.start_date = d.calday
  WHERE
    business_unit IN ('1500','2000')
    AND mch2_id IN ('102','201','202','203')
)
,summary AS (
  SELECT
    a.* ,
    CASE 
    WHEN leadtime_actual IS NULL THEN 
        CASE 
            WHEN IF(GREATEST(0, leadtime_actual) = 0, NULL, GREATEST(0, leadtime_actual)) <= leadtime_qual THEN 'ONTIME'
            WHEN start_date > end_date THEN 'NOT COUNT'
            WHEN time_tilltoday <= leadtime_qual THEN 'IN LEADTIME REQUIRED'
            ELSE 'LATE'
        END
    WHEN GREATEST(0, leadtime_actual) <= leadtime_qual THEN 'ONTIME'
    WHEN start_date > end_date THEN 'NOT COUNT'
    WHEN time_tilltoday <= leadtime_qual THEN 'IN LEADTIME REQUIRED'
    ELSE 'LATE'
    END AS ontime_check
  FROM ontime_check a
)
SELECT a.* ,
    IF(end_date IS NOT NULL,gi_qty,0) infull_qty,
    IF(ontime_check = 'ONTIME',gi_qty,0) ontime_qty
FROM summary a;
""")